In [1]:
!pip install transformers onnx onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!python -m transformers.onnx --model=bert-base-uncased model

2022-07-23 03:08:59.690568: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification m

In [3]:
import onnx
from onnxruntime.quantization import quantize_dynamic

model_fp32 = 'model/model.onnx'
model_quant = 'model/model.quantized.onnx'
quantize_dynamic(model_fp32, model_quant)

Ignore MatMul due to non constant B: /[MatMul_149]
Ignore MatMul due to non constant B: /[MatMul_154]
Ignore MatMul due to non constant B: /[MatMul_243]
Ignore MatMul due to non constant B: /[MatMul_248]
Ignore MatMul due to non constant B: /[MatMul_337]
Ignore MatMul due to non constant B: /[MatMul_342]
Ignore MatMul due to non constant B: /[MatMul_431]
Ignore MatMul due to non constant B: /[MatMul_436]
Ignore MatMul due to non constant B: /[MatMul_525]
Ignore MatMul due to non constant B: /[MatMul_530]
Ignore MatMul due to non constant B: /[MatMul_619]
Ignore MatMul due to non constant B: /[MatMul_624]
Ignore MatMul due to non constant B: /[MatMul_713]
Ignore MatMul due to non constant B: /[MatMul_718]
Ignore MatMul due to non constant B: /[MatMul_807]
Ignore MatMul due to non constant B: /[MatMul_812]
Ignore MatMul due to non constant B: /[MatMul_901]
Ignore MatMul due to non constant B: /[MatMul_906]
Ignore MatMul due to non constant B: /[MatMul_995]
Ignore MatMul due to non consta

In [4]:
!ls -alh model

total 523M
drwxr-xr-x 2 root root 4.0K Jul 23 02:51 .
drwxr-xr-x 1 root root 4.0K Jul 23 02:50 ..
-rw-r--r-- 1 root root 418M Jul 23 03:09 model.onnx
-rw-r--r-- 1 root root 105M Jul 23 03:09 model.quantized.onnx


In [5]:
import time
import onnxruntime as ort
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
encoded_input = tokenizer(" ".join(["hello"]*510), return_tensors='np')

session = ort.InferenceSession("model/model.onnx", providers=['CPUExecutionProvider'])
quantized_session = ort.InferenceSession("model/model.quantized.onnx", providers=['CPUExecutionProvider'])

def benchmark(sess):
  start = time.time()
  for _ in range(10):
    output = sess.run(None, input_feed=dict(encoded_input))
  end = time.time()
  print(end - start)

benchmark(session)
benchmark(quantized_session)

17.39389729499817
12.877118349075317
